In [1]:
import os
import pandas as pd
from ast import literal_eval
from typing import List
from IPython.core.display import HTML
import string
import streamlit as st
import pandas as pd
import numpy as np
from string import punctuation
from nltk.tokenize import sent_tokenize

from itertools import groupby
from operator import itemgetter
from copy import copy
import warnings
warnings.filterwarnings('ignore')

In [2]:
def delete_context(text: str) -> str:
    if text[0] == "[":
        n_strings = len(text)
        hook_end = 0
        while text[hook_end] != "]" and hook_end < 50:
            hook_end += 1
            if hook_end==(n_strings-1):
                return text

        if hook_end < 50:
            return text[(hook_end + 1) :].lstrip()
        else:
            return text.lstrip()

    else:
        return text.lstrip()

In [3]:
DATA_PATH = os.path.join(
    "..", "..", "..", "..", "data", "frameworks_data", "data_v0.7.1"
)

test_df = pd.read_csv(
    os.path.join(DATA_PATH, "new_columns_test_v0.7.1.csv.gz"), compression="gzip"
).drop_duplicates()
#test_df['target'] = test_df['target'].apply(literal_eval)

test_df = test_df[(test_df.lang=='en') & (test_df.target.apply(lambda x: len(x)>0))].drop(columns=['lang'])
test_df['excerpt'] = test_df['excerpt'].apply(delete_context)
test_df['excerpt'] = test_df['excerpt'].apply(sent_tokenize)
test_df = test_df.explode('excerpt')
test_df['len_excerpt'] = test_df['excerpt'].apply(lambda x: len(x.split(' ')))
test_df = test_df[test_df.len_excerpt<30]

# For LGBTQI + to become LGBTQI+
test_df['excerpt'] = test_df['excerpt'].apply(lambda x: x.replace(' +', '+'))

test_df.shape

(25028, 6)

In [4]:
specific_groups_kw = ["homosex", "lgbt", "lgtb", "lesbian", "bisex", "transgender", "intersex", 
"queer", "sex worker"]#, "refugee", "migrant", "asylum", 'idp', 'internally displaced people']

test_df['specific_groups_kw'] = test_df.excerpt.apply(lambda x: [one_kw for one_kw in specific_groups_kw if one_kw in x.lower()])
specific_groups_df = test_df[test_df.specific_groups_kw.apply(lambda x: len(x)>=1)]
print(specific_groups_df.columns)
print(specific_groups_df.shape)

Index(['excerpt', 'entry_id', 'target', 'analysis_framework_id', 'project_id',
       'len_excerpt', 'specific_groups_kw'],
      dtype='object')
(38, 7)


In [ ]:
specific_groups_df[['excerpt', 'entry_id', 'specific_groups_kw']].to_csv('prepare_data_streamlit/specific_groups.csv', index=None)

#### We enter the df to the streamlit app and label each excerpt

In [5]:
annotated_specific_groups = pd.read_csv('final_data/minorities_df_tmp.csv', index_col=0)
annotated_specific_groups.head()

FileNotFoundError: [Errno 2] No such file or directory: 'final_data/minorities_df_tmp.csv'

In [6]:
def clean_lgbt_words(text: str) -> str:
    def lgbt_in_word(word):
        return "lgbt" in word.lower() or "lgtb" in word.lower()

    def get_one_lgbt_token(word):
        if word[-1] in punctuation and word[-1] != "+":
            final_punct = word[-1]
            word = word[:-1]
        else:
            final_punct = ""

        return ("lgbt" + final_punct).rstrip()

    clean_text = copy(text)
    if lgbt_in_word(clean_text):
        clean_text = clean_text.replace(" +", "+")
        words = clean_text.split(" ")
        output_text = " ".join(
            [
                one_word if not lgbt_in_word(one_word) else get_one_lgbt_token(one_word)
                for one_word in words
            ]
        )
        return output_text

    else:
        return text

In [129]:
def posptprocess_relevancy_list(relevancy_list):
    posptrocessed = list(set(relevancy_list))
    if len(posptrocessed)>1:
        posptrocessed = [item for item in posptrocessed if 'no' in str(item)]
    return posptrocessed[0]

relevancy_col = annotated_specific_groups.relevant.tolist()
ids_col = annotated_specific_groups.entry_id.tolist()
n_tot_rows = len(relevancy_col)
treated_rows = [i for i, number in enumerate(ids_col) if str(number) != "nan"]

ranges = []

for k, g in groupby(enumerate(treated_rows), lambda x: x[0] - x[1]):
    group = map(itemgetter(1), g)
    group = list(map(int, group))
    ranges.append(group)

relevancy_items = [
    posptprocess_relevancy_list([relevancy_col[item] for item in sublist]) for sublist in ranges
]

specific_groups_df = pd.read_csv('prepare_data_streamlit/specific_groups.csv')
specific_groups_df['relevancy'] = relevancy_items
specific_groups_df = specific_groups_df[specific_groups_df.relevancy!='n']
specific_groups_df['specific_groups_kw'] = specific_groups_df['specific_groups_kw'].apply(
    lambda x: [item for item in literal_eval(x) if item!='homosex'][0].replace('lgtb', 'lgbt')
)
specific_groups_df.to_csv('tmp_change_spec_groups.csv', index=None)

In [9]:
specific_groups_df = pd.read_csv('tmp_change_spec_groups.csv').drop(columns=['relevancy']).rename(
    columns={'specific_groups_kw': 'kw'}
)
specific_groups_df['excerpt'] = specific_groups_df['excerpt'].apply(clean_lgbt_words)
specific_groups_df['type'] = 'original'
specific_groups_df.head()

,excerpt,entry_id,kw,type
0,Feedback suggests that Adolescent sex workers ...,233960.0,sex worker,original
1,Some sex workers say police and local people s...,233960.0,sex worker,original
2,In Puerto Lleras the FGD reveals that illegal ...,165886.0,lgbt,original
3,"Finally, the pandemic has disproportionally, a...",165180.0,lgbt,original
4,The property houses a diverse profile includin...,166343.0,lgbt,original


In [10]:
specific_groups_df.shape

(21, 4)

In [12]:
specific_groups_kw = [
    "transgender",
    "lgbt",
    "sex worker",
    "homosexual",
    "lesbian",
    "bisexual",
    "intersexual",
    "queer",
    "refugee",
    "migrant",
    "asylum seeker",
    "person",
    'rhoringya'
]

final_specific_groups_df = pd.DataFrame()
for i in range(specific_groups_df.shape[0]):
    row = specific_groups_df.iloc[[i]]
    original_row_kw = row["kw"].values[0]
    row["type"] = "augmented"
    for one_kw in specific_groups_kw:
        if one_kw != original_row_kw:
            row_tmp = row.copy()
            row_tmp["excerpt"] = row_tmp["excerpt"].apply(
                lambda x: x.replace(original_row_kw, one_kw).replace(
                    original_row_kw.capitalize(), one_kw.capitalize()
                )
            )
            row_tmp["kw"] = [one_kw]
            final_specific_groups_df = final_specific_groups_df.append(row_tmp)

final_specific_groups_df = pd.concat(
    [specific_groups_df, final_specific_groups_df]
)

final_specific_groups_df

,excerpt,entry_id,kw,type
0,Feedback suggests that Adolescent sex workers ...,233960.0,sex worker,original
1,Some sex workers say police and local people s...,233960.0,sex worker,original
2,In Puerto Lleras the FGD reveals that illegal ...,165886.0,lgbt,original
3,"Finally, the pandemic has disproportionally, a...",165180.0,lgbt,original
4,The property houses a diverse profile includin...,166343.0,lgbt,original
...,...,...,...,...
20,They indicated that the level of tolerance tow...,57176.0,migrant,augmented
20,They indicated that the level of tolerance tow...,57176.0,asylum seeker,augmented
20,They indicated that the level of tolerance tow...,57176.0,IDP,augmented
20,They indicated that the level of tolerance tow...,57176.0,internally displaced people,augmented


In [13]:
final_specific_groups_df.shape[0] / specific_groups_df.shape[0]

14.0

In [14]:
final_specific_groups_df.to_csv('final_data/test_minorities_df.csv', index=None)

## Gender

In [20]:
female_kw = ["women", "woman", "girl", "female"]#, "refugee", "migrant", "asylum", 'idp', 'internally displaced people']
male_kw = ["men", "man", "male", 'boy']

def men_kw_in_excerpt(excerpt: str) -> bool:
    low_excerpt = excerpt.lower()
    if 'boy' in low_excerpt:
        return True
    
    excerpt_words = low_excerpt.split(' ')
    for one_word in excerpt_words:
        one_word_no_punctuation = one_word.translate(str.maketrans('', '', string.punctuation))
        if any([one_word_no_punctuation==one_male_kw for one_male_kw in ["men", "man", "male"]]):
            return True

    return False

test_df['gender_kw'] = test_df.excerpt.apply(lambda x: [one_kw for one_kw in female_kw if one_kw in x.lower()])
gender_df = test_df[(test_df.gender_kw.apply(lambda x: len(x)>1)) & (test_df.excerpt.apply(lambda x: not men_kw_in_excerpt(x)))]
print(gender_df.columns)
print(gender_df.shape)

Index(['excerpt', 'entry_id', 'target', 'analysis_framework_id', 'project_id',
       'len_excerpt', 'gender_kw'],
      dtype='object')
(150, 7)


In [44]:
gender_df.head()

,excerpt,entry_id,target,analysis_framework_id,project_id,len_excerpt,gender_kw
77,Impact on women and girls As in many other cou...,186577.0,"['first_level_tags->sectors->Health', 'first_l...",829.0,1184.0,20,"[women, girl]"
331,Some women and girls noted that they experienc...,219352.0,"['first_level_tags->sectors->Protection', 'fir...",1306.0,2028.0,22,"[women, girl]"
929,Women also reported shortages of water in camp...,490712.0,"['first_level_tags->sectors->WASH', 'first_lev...",1306.0,2170.0,23,"[women, girl]"
1319,"Following the training, the women’s committee ...",169626.0,"['first_level_tags->sectors->Health', 'first_l...",1306.0,2098.0,24,"[women, girl]"
1905,A female participant stated that women would c...,203102.0,"['first_level_tags->sectors->Protection', 'fir...",1306.0,2028.0,18,"[women, female]"


In [23]:
gender_df[['excerpt', 'entry_id', 'gender_kw']].to_csv('prepare_data_streamlit/gender.csv', index=None)


In [33]:
female_kw = ["women", "woman", "girl", "female", "mother"]#, "refugee", "migrant", "asylum", 'idp', 'internally displaced people']
male_kw = ["men", "man", "male", 'boy', 'boys', "father", "fathers"]
female_linked_lexic = ['menstr', 'pregnan', 'hygien', 'lacta']

def men_kw_in_excerpt(excerpt: str) -> List[str]:
    men_kw = []
    low_excerpt = excerpt.lower()    
    excerpt_words = low_excerpt.split(' ')
    for one_word in excerpt_words:
        one_word_no_punctuation = one_word.translate(str.maketrans('', '', string.punctuation))
        men_kw += [one_male_kw for one_male_kw in male_kw if one_word_no_punctuation==one_male_kw]

    return men_kw

def kw_in_excerpt(excerpt: str, kw_list: List[str]) -> List[str]:
    low_excerpt = excerpt.lower()
    return [one_female_kw for one_female_kw in kw_list if one_female_kw in low_excerpt]


test_df['male_kw'] = test_df.excerpt.apply(men_kw_in_excerpt)
test_df['female_kw'] = test_df.excerpt.apply(lambda x: kw_in_excerpt(x, female_kw))
test_df['female_lexic_kw'] = test_df.excerpt.apply(lambda x: kw_in_excerpt(x, female_linked_lexic))

both_genders_df = test_df[(test_df.male_kw.apply(lambda x: len(x)>0)) | (test_df.female_kw.apply(lambda x: len(x)>0))]
print(both_genders_df.columns)
print(both_genders_df.shape)

Index(['excerpt', 'entry_id', 'target', 'analysis_framework_id', 'project_id',
       'len_excerpt', 'male_kw', 'female_kw', 'female_lexic_kw'],
      dtype='object')
(1552, 9)


In [34]:
#both_genders_df[['excerpt', 'entry_id', 'male_kw', 'female_lexic_kw', "female_kw"]].to_csv('final_data/gender_df_tmp.csv', index=None)

In [35]:
original_gender_df = both_genders_df[
    both_genders_df.female_lexic_kw.apply(lambda x: len(x) == 0)
].drop(
    columns=[
        "analysis_framework_id",
        "project_id",
        "female_lexic_kw",
        "len_excerpt",
    ]
)

original_gender_df = original_gender_df[~(
    (original_gender_df.male_kw.apply(lambda x: len(x) > 0))
    & (original_gender_df.female_kw.apply(lambda x: len(x) > 0)))
]

original_gender_df.head()
original_gender_df['type'] = 'original'
original_gender_df['kw'] = original_gender_df.apply(
    lambda x: x['male_kw'] if len(x['male_kw'])>0 else x['female_kw'], axis=1
)


In [36]:
original_gender_df.columns, original_gender_df.shape

(Index(['excerpt', 'entry_id', 'target', 'male_kw', 'female_kw', 'type', 'kw'], dtype='object'),
 (1087, 7))

In [37]:
from copy import copy

mapping_male_to_female = {
    "men": "women",
    "man": "woman",
    "male": "female",
    "boy": "girl",
    "father": "mother",
}
mapping_female_to_male = {v: k for k, v in mapping_male_to_female.items()}

mapping_male_to_neutral = {k: 'person' for k in mapping_male_to_female.keys()}
mapping_female_to_neutral = {v: 'person' for v in mapping_male_to_female.values()}

def replace_gender_kw(excerpt, mapping):
    excerpt_tmp = copy(excerpt)
    for one_male_kw, one_female_kw in mapping.items():
        excerpt_tmp = excerpt_tmp.replace(one_male_kw, one_female_kw)
        excerpt_tmp = excerpt_tmp.replace(
            one_male_kw.capitalize(), one_female_kw.capitalize()
        )
    return excerpt_tmp

augmented_gender_df = original_gender_df.copy()
augmented_gender_df["type"] = "augmented"

for mapping_dict in [mapping_male_to_female, mapping_male_to_neutral]:
    # change male to female
    augmented_male_df = augmented_gender_df[
        augmented_gender_df.male_kw.apply(lambda x: len(x) > 0)
    ]
    augmented_male_df["excerpt"] = augmented_male_df["excerpt"].apply(
        lambda x: replace_gender_kw(x, mapping_dict)
    )
    augmented_male_df["kw"] = augmented_male_df["kw"].apply(
        lambda x: [
            replace_gender_kw(item.replace("s", ""), mapping_dict) for item in x
        ]
    )
    original_gender_df = original_gender_df.append(augmented_male_df)


# change female to male
for mapping_dict in [mapping_female_to_male, mapping_female_to_neutral]:

    augmented_female_df = augmented_gender_df[
        augmented_gender_df.female_kw.apply(lambda x: len(x) > 0)
    ]
    augmented_female_df["excerpt"] = augmented_female_df["excerpt"].apply(
        lambda x: replace_gender_kw(x, mapping_dict)
    )
    augmented_female_df["kw"] = augmented_female_df["kw"].apply(
        lambda x: [replace_gender_kw(item, mapping_dict) for item in x]
    )
    original_gender_df = original_gender_df.append(augmented_female_df)

final_gender_df = original_gender_df[['entry_id', 'excerpt', 'kw', 'type']].sort_values(by=['entry_id', 'type'])

In [38]:
final_gender_df.kw.apply(str).unique()

array(["['girl']", "['person']", "['boys']", "['male']", "['female']",
       "['men']", "['women']", "['boy']", "['father']", "['mother']",
       "['man']", "['woman']", "['women', 'girl']",
       "['person', 'person']", "['men', 'boys']", "['men', 'boy']",
       "['women', 'women']", "['men', 'men']", "['boy', 'father']",
       "['girl', 'mother']", "['men', 'male']", "['women', 'female']",
       "['men', 'man']", "['women', 'woman']", "['men', 'boy', 'male']",
       "['person', 'person', 'person']", "['women', 'girl', 'female']",
       "['boy', 'male']", "['girl', 'female']", "['female', 'female']",
       "['male', 'male']", "['men', 'father']", "['women', 'mother']"],
      dtype=object)

In [39]:
final_gender_df.to_csv('final_data/test_gender_df.csv', index=None)

## Countries

In [ ]:
countries_kw = [
    "colombia", "bogot", "venezuel", "syri", "banglad", "hondur", "niger", "damasc", "congo", "sudan", "salvador", "Ecuador",
    "tanganyika", "peru", "nicaragua", "libya", "afghanistan", "costa rica", 
]

In [7]:
countries_kw = [
    "congo",
    "lebanon",
    "syria",
    "peru",
    "niger",
    "venezue",
    "sudan",
    "turkey",
    "brazil",
    "brasil",
    "burundi",
    "nicaragua",
    "africa",
    "europe",
    "bangladesh",
    "argentina",
    "uruguay",
    "iran",
    "birmingham",
    "salvador",
    "somalia",
    "bogot",
    "australia",
    "myanmar",
    "guatemala",
    "pakistan",
    "china",
    "ethiopia",
    "honduras",
    "swed",
    "damasc",
    "serbia",
    "ecuador",
    "nigeria",
    "mali",
    "rohinga",
    "bosni",
    "iraq",
    "trinidad and tobago",
    "cox's bazar",
    "beiruth",
    "algeria",
    "libya",
    "colombia",
    "india",
    "rwanda",
    "united states",
    "cameroon",
    "yemen",
    "canada",
    "france",
    "italy",
    "romania",
    "afghanistan",
    "cameroon",
    "paraguay"
]

countries_df = test_df.copy()

countries_df["kw"] = countries_df.excerpt.apply(
    lambda x: [{one_country: word} for one_country in countries_kw for word in x.lower().split(' ') if one_country in word]
)

countries_df = countries_df[countries_df.kw.apply(lambda x: len(x)==1)]
countries_df

,excerpt,entry_id,target,analysis_framework_id,project_id,len_excerpt,kw
3,The Migration Policy Institute also estimates ...,187037.0,['first_level_tags->pillars_1d->Humanitarian A...,829.0,1184.0,19,[{'ecuador': 'ecuador'}]
42,"In Ecuador, only 51.9 per cent of children und...",187553.0,"['first_level_tags->sectors->WASH', 'first_lev...",829.0,1184.0,20,"[{'ecuador': 'ecuador,'}]"
51,The areas in northeastern Syria did not receiv...,243576.0,"['first_level_tags->sectors->Health', 'first_l...",1306.0,2028.0,21,[{'syria': 'syria'}]
77,Women constitute 60 percent of general health ...,186577.0,"['first_level_tags->sectors->Health', 'first_l...",829.0,1184.0,25,"[{'ecuador': 'ecuador,'}]"
81,All 299 arrivals were from Diffa region in Nig...,187327.0,"['first_level_tags->pillars_1d->Displacement',...",1306.0,2170.0,10,[{'niger': 'niger'}]
...,...,...,...,...,...,...,...
23655,"For Ecuadorian households, the main difference...",102504.0,[],273.0,515.0,17,[{'ecuador': 'ecuadorian'}]
23660,• The Department of Nariño ranks 18 in relatio...,188418.0,[],829.0,1183.0,16,[{'venezue': 'venezuelans'}]
23660,• 41% of Venezuelans in Nariño are located in ...,188418.0,[],829.0,1183.0,17,[{'venezue': 'venezuelans'}]
23669,"Incursions and violent incidents in Niger, Cha...",209469.0,[],1465.0,2334.0,18,"[{'niger': 'niger,'}]"


In [8]:
countries_df[['entry_id', 'excerpt','kw']].to_csv('locations_nationalities.csv', index=None)